In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train_x =train.drop(['Survived'], axis=1)
train_y = train['Survived']
test_x = test.copy()

train_x = train_x.drop(['PassengerId'], axis=1)
test_x = test_x.drop(['PassengerId'], axis=1)

train_x = train_x.drop(['Name', 'Ticket', 'Cabin'], axis=1)
test_x = test_x.drop(['Name', 'Ticket', 'Cabin'], axis=1)

num_cols = [col for col in train_x.columns if train_x[col].dtype != 'object']
cat_cols = [col for col in train_x.columns if train_x[col].dtype == 'object']

print(num_cols)
print(cat_cols)
train_x.head()

['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
['Sex', 'Embarked']


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [3]:
for col in cat_cols:
    train_x[col] = train_x[col].astype('category')
    test_x[col] = test_x[col].astype('category')

In [4]:
tr_x, va_x, tr_y, va_y = train_test_split(train_x, train_y, test_size=0.2, random_state=22)

In [5]:
lgb_train = lgb.Dataset(tr_x, tr_y)
lgb_eval = lgb.Dataset(va_x, va_y)

params = {'objective': 'binary', 'seed': 71, 'verbose': 0, 'metrics': 'binary_logloss'}
num_round = 100

categorical_features = cat_cols

model = lgb.train(
    params,
    lgb_train,
    num_boost_round=num_round,
    categorical_feature=categorical_features,
    valid_names=['train', 'valid'],
    valid_sets=[lgb_train, lgb_eval]
)

va_pred = model.predict(va_x)
score = log_loss(va_y, va_pred)
print(f'logloss: {score:.4f}')

pred = model.predict(test_x)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001010 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	train's binary_logloss: 0.615954	valid's binary_logloss: 0.628564
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	train's binary_logloss: 0.57613	valid's binary_logloss: 0.599518
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	train's binary_logloss: 0.543876	valid's binary_logloss: 0.574622
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	train's binary_logloss: 0.516471	valid's binary_logloss: 0.554041
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	train's binary_logloss: 0.492632	valid's binary_logloss: 0.537404
[LightGBM] [Warning] No further splits with pos

/Library/Python/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1554: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Embarked', 'Sex']
  warnings.warn('categorical_feature in Dataset is overridden.\n'
/Library/Python/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/Library/Python/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))
